In [ ]:
!pip install kaggle

In [ ]:
#print(tensorflow)
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [ ]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [ ]:
#!kaggle competitions list

In [ ]:
!kaggle competitions download -c tweet-sentiment-extraction 

  0% 0.00/307k [00:00<?, ?B/s]
100% 307k/307k [00:00<00:00, 46.2MB/s]
  0% 0.00/41.4k [00:00<?, ?B/s]
100% 41.4k/41.4k [00:00<00:00, 43.1MB/s]
  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 83.5MB/s]


In [ ]:
!pip install transformers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.6MB/s 
     |████████████████████████████████| 3.8MB 18.8MB/s 
     |████████████████████████████████| 890kB 42.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b4cd874ee2e1747a247190a6512cd2f20afe3e9390989015065d73505ae5a7d6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns; sns.set(style='white')
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import tensorflow.keras.backend as K
from transformers import *
import tokenizers
from sklearn.model_selection import StratifiedKFold
import math
import re
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import concatenate

In [ ]:
MAX_LEN = 96
PATH = 'drive/My Drive/tf-roberta/'
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=PATH+'vocab-roberta-base.json', 
    merges_file=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
EPOCHS = 3 # originally 3
BATCH_SIZE = 32 # originally 32
PAD_ID = 1
SEED = 88888
LABEL_SMOOTHING = 0.1
tf.random.set_seed(SEED)
np.random.seed(SEED)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv('drive/My Drive/tw_senti_com/train.csv').fillna('')
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
#先頭なら問答無用でそのまま出力
#先頭なら後ろにスペースがあってもそのまま出力。
def preprocess_txt(text1, text2):
  sp_loc = text1.find('  ')
  comma_loc = text1.find('`')
  sel_loc = text1.find(text2)
  if sp_loc != -1 and sp_loc <= sel_loc + 1:
    if len(text2.split()[0]) == 1:
      return text2.strip()[1:].strip()
  #elif comma_loc != -1 and comma_loc <= sel_loc + 1:
    #if len(text2.strip().split()[0]) != 0:
      #return text2.strip()[1:]
    else:
      return text2
  else:
    return text2
#シャープ版も作ってね!
#¿½も
#gooはgoodに変換する。
#単語の途中で始まるものは左の一語の一文字だけ取り除く
#選択したテキストないの'  ' 

In [ ]:
preprocess_txt('i like  curry rice very much', 'y much')

'much'

In [ ]:
import re 
def preprocess_all_tw(txt):
  txt = txt.strip()
  return re.sub(r'\ {1,}', r' ', txt)

text_sam = ' i dont like  curry   rice'
preprocess_all_tw(text_sam)

'i dont like curry rice'

In [ ]:
import random
import pandas as pd
from nltk.corpus import words
import nltk
nltk.download('all')

In [ ]:
df = pd.read_csv("train.csv.zip")
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
test = pd.read_csv('test.csv').fillna('')

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    all_tweet_test = test.loc[k,'text']
    all_tweet_test = preprocess_all_tw(all_tweet_test)
    text1 = " "+" ".join(all_tweet_test.split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+3] = [0, s_tok] + enc.ids + [2]
    attention_mask_t[k,:len(enc.ids)+3] = 1

In [ ]:
Dropout_new = 0.15     # originally 0.1
n_split = 5            # originally 5
lr = 3e-5              # originally 3e-5

In [ ]:
import pickle

def save_weights(model, dst_fn):
    weights = model.get_weights()
    with open(dst_fn, 'wb') as f:
        pickle.dump(weights, f)


def load_weights(model, weight_fn):
    with open(weight_fn, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)
    return model

def loss_fn(y_true, y_pred):
    # adjust the targets for sequence bucketing
    ll = tf.shape(y_pred)[1]
    y_true = y_true[:, :ll]
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=LABEL_SMOOTHING)
    loss = tf.reduce_mean(loss)
    return loss

def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(Dropout_new)(x[0])
    x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Conv1D(64, 2,padding='same')(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(Dropout_new)(x[0]) 
    x2 = tf.keras.layers.Conv1D(768, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(64, 2, padding='same')(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    #'右'だけpadで埋める
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model    

In [ ]:
import pickle

def save_weights(model, dst_fn):
    weights = model.get_weights()
    with open(dst_fn, 'wb') as f:
        pickle.dump(weights, f)

def load_weights(model, weight_fn):
    with open(weight_fn, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)
    return model

def loss_fn(y_true, y_pred):
    # adjust the targets for sequence bucketing
    ll = tf.shape(y_pred)[1]
    y_true = y_true[:, :ll]
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=LABEL_SMOOTHING)
    loss = tf.reduce_mean(loss)
    return loss

def build_model_lstm():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)

    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    
    x1 = tf.keras.layers.Dropout(Dropout_new)(x[0])
    x1 = Bidirectional(CuDNNLSTM(784, return_sequences=True, name='lstm_layer',))(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(Dropout_new)(x[0])
    x2 = Bidirectional(CuDNNLSTM(784, return_sequences=True, name='lstm_layer',))(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    
    padded_model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model    

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train.shape[0]):
    
    # FIND OVERLAP
    all_tweet = train.loc[k,'text']
    sel_tweet_bef = train.loc[k,'selected_text']
    sel_tweet = preprocess_txt(all_tweet, sel_tweet_bef)
    #'  'を探した後に'  'を削除しないとfindがズレる。
    #all_tweet = preprocess_all_tw(all_tweet)

    text1 = " "+" ".join(all_tweet.split())
    text2 = " ".join(sel_tweet.split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
       # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+3] = [0, s_tok] + enc.ids + [2]
    attention_mask[k,:len(enc.ids)+3] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+2] = 1
        end_tokens[k,toks[-1]+2] = 1 

In [ ]:
%%time
jac = []; VER='v0'; DISPLAY=1 
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start_train = np.zeros((input_ids.shape[0],MAX_LEN))
preds_end_train = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=n_split,shuffle=True,random_state=SEED)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model, padded_model = build_model_lstm()
        
    inpT = [input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]]
    targetT = [start_tokens[idxT,], end_tokens[idxT,]]
    inpV = [input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]]
    targetV = [start_tokens[idxV,], end_tokens[idxV,]]

    #cvが小さい方が文字数が少なく、cvが大きい方が文字数が多い
    #要はpadが大きい順にソート(文字数が小さい順にソート)
    shuffleV = np.int32(sorted(range(len(inpV[0])), key=lambda k: (inpV[0][k] == PAD_ID).sum(), reverse=True))
    inpV = [arr[shuffleV] for arr in inpV]
    targetV = [arr[shuffleV] for arr in targetV]
    
    weight_fn = 'drive/My Drive/roberta_lstm2/%s-roberta-%i.h5'%(VER,fold)

    for epoch in range(1, EPOCHS + 1):
        # PAD_IDの数でソート(上のセルを実行してからshuffleを確認)
        shuffleT = np.int32(sorted(range(len(inpT[0])), key=lambda k: (inpT[0][k] == PAD_ID).sum() + np.random.randint(-3, 3), reverse=True))
        # shuffle in batches, kjtherwise short batches will always come in the beginning of each epoch
        num_batches = math.ceil(len(shuffleT) / BATCH_SIZE)
        batch_inds = np.random.permutation(num_batches)
        shuffleT_ = []
        for batch_ind in batch_inds:
            shuffleT_.append(shuffleT[batch_ind * BATCH_SIZE: (batch_ind + 1) * BATCH_SIZE])
        shuffleT = np.concatenate(shuffleT_)
        inpT = [arr[shuffleT] for arr in inpT]
        targetT = [arr[shuffleT] for arr in targetT]
        model.fit(inpT, targetT, 
            epochs=epoch, initial_epoch=epoch - 1, batch_size=BATCH_SIZE, verbose=DISPLAY, callbacks=[],
            validation_data=(inpV, targetV), shuffle=False)  # don't shuffle 
        save_weights(model, weight_fn)
    
    print('Loading model...')
    load_weights(model, weight_fn)

    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = padded_model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting all Train for Outlier analysis...')
    preds_train = padded_model.predict([input_ids,attention_mask,token_type_ids],verbose=DISPLAY)
    preds_start_train += preds_train[0]/skf.n_splits
    preds_end_train += preds_train[1]/skf.n_splits
    
    print('Predicting Test...')
    preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] #普通はa<bだが、確率的にそうでない場合もあるから、その場合をハンドリング
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-2:b-1])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()

#########################
### FOLD 1
#########################
687/687 [==============================] - 143s 209ms/step - loss: 2.8561 - activation_loss: 1.4138 - activation_1_loss: 1.4423 - val_loss: 2.5289 - val_activation_loss: 1.2584 - val_activation_1_loss: 1.2704
Epoch 2/2
687/687 [==============================] - 113s 165ms/step - loss: 2.5129 - activation_loss: 1.2568 - activation_1_loss: 1.2561 - val_loss: 2.4986 - val_activation_loss: 1.2483 - val_activation_1_loss: 1.2503
Epoch 3/3
687/687 [==============================] - 114s 166ms/step - loss: 2.3950 - activation_loss: 1.2023 - activation_1_loss: 1.1927 - val_loss: 2.4736 - val_activation_loss: 1.2356 - val_activation_1_loss: 1.2380
Loading model...
Predicting OOF...
172/172 [==============================] - 38s 223ms/step
Predicting all Train for Outlier analysis...
859/859 [==============================] - 92s 107ms/step
Predicting Test...
111/111 [==============================] - 22s 198ms/step
>>>> FOLD 1 Jacca

In [ ]:
print('>>5Fold CV Jaccard =',np.mean(jac))
print(jac) # Jaccard CVs

>>5Fold CV Jaccard = 0.704918095854755
[0.7070342347986956, 0.7098534367766568, 0.7087365644999007, 0.6950993396102888, 0.7038669035882328]


In [ ]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-2:b-1])
    all.append(st)

In [ ]:
test_raw = pd.read_csv('test.csv').fillna('')

In [ ]:
print(test.shape, test_raw.shape)

NameError: ignored

In [ ]:
test['selected_text'] = all

NameError: ignored

In [ ]:
import re
def rm_url(text): 
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    return ''.join(text)
def rm_underbar(text):
    text = re.sub('^_.{2,}?\s', '', text)
    return ''.join(text)

In [ ]:
test_raw = pd.read_csv('/content/test.csv').fillna('')
for k in range(test.shape[0]):
    if test.loc[k, 'sentiment'] == 'positive' or 'negative':
        if len(test.loc[k, 'text'].split()) <= 2:
            test.loc[k, 'selected_text'] = test_raw.loc[k, 'text']
    #if test.loc[k, 'sentiment'] == 'neutral': 
        #test.loc[k, 'selected_text'] = test_raw.loc[k, 'text']

test['selected_text'] = test['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
test['selected_text'] = test['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
test['selected_text'] = test['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)
#test['selected_text'] = test['selected_text'].apply(lambda x: rm_url(x))

In [ ]:
test[['textID','selected_text']].to_csv('submission.csv',index=False)
test.sample(10)

,textID,text,sentiment,selected_text
3444,7d32d4866a,You are sooo lucky. My fiance is away w/the M...,positive,lucky.
749,200f2b3566,you shouldnt have to reset more than once if...,neutral,you shouldnt have to reset more than once if ...
3136,43c61eb7a3,I can`t find my camera,negative,i can`t find my camera
26,334954f215,"hey peoples, dont you just hate being grounded...",neutral,"hey peoples, dont you just hate being grounde..."
3208,2dfbe0b7fb,Hmmm. http://www.djhero.com/ is down,negative,down
2095,8986091fd3,wishes happy mothers` day to all moms out there.,positive,happy
843,7dd51bdec1,reading Evermore by Lynn Viehl.,neutral,reading evermore by lynn viehl.
172,5ff525b74b,Wondering if i cld make things any worse than ...,negative,worse
100,f9382a1f7b,"oooh, sunshine! A patch of sunshine! And it wi...",neutral,"oooh, sunshine! a patch of sunshine! and it w..."
930,7720aca199,"No, that`s not right - I remember now. You w...",neutral,"no, that`s not right - i remember now. you we..."


In [ ]:
pytorch_score = '''
Fold: 1
Epoch 1/3 | train | Loss: 2.1321 | Jaccard: 0.6634
Epoch 1/3 |  val  | Loss: 1.6480 | Jaccard: 0.7166
Epoch 2/3 | train | Loss: 1.6013 | Jaccard: 0.7182
Epoch 2/3 |  val  | Loss: 1.6582 | Jaccard: 0.7059
Epoch 3/3 | train | Loss: 1.4530 | Jaccard: 0.7373
Epoch 3/3 |  val  | Loss: 1.7208 | Jaccard: 0.7131
Fold: 2
Epoch 1/3 | train | Loss: 2.1968 | Jaccard: 0.6553
Epoch 1/3 |  val  | Loss: 1.7515 | Jaccard: 0.6925
Epoch 2/3 | train | Loss: 1.6998 | Jaccard: 0.7068
Epoch 2/3 |  val  | Loss: 1.6465 | Jaccard: 0.7124
Epoch 3/3 | train | Loss: 1.5439 | Jaccard: 0.7252
Epoch 3/3 |  val  | Loss: 1.6395 | Jaccard: 0.7160
Fold: 3
Epoch 1/3 | train | Loss: 2.1237 | Jaccard: 0.6675
Epoch 1/3 |  val  | Loss: 1.7006 | Jaccard: 0.7078
Epoch 2/3 | train | Loss: 1.6322 | Jaccard: 0.7172
Epoch 2/3 |  val  | Loss: 1.6523 | Jaccard: 0.7141
Epoch 3/3 | train | Loss: 1.4792 | Jaccard: 0.7337
Epoch 3/3 |  val  | Loss: 1.6673 | Jaccard: 0.7169
Fold: 4
Epoch 1/3 | train | Loss: 2.1617 | Jaccard: 0.6601
Epoch 1/3 |  val  | Loss: 1.5938 | Jaccard: 0.7212
Epoch 2/3 | train | Loss: 1.6270 | Jaccard: 0.7123
Epoch 2/3 |  val  | Loss: 1.5249 | Jaccard: 0.7308
Epoch 3/3 | train | Loss: 1.4600 | Jaccard: 0.7373
Epoch 3/3 |  val  | Loss: 1.5597 | Jaccard: 0.7247
Fold: 5
Epoch 1/3 | train | Loss: 2.0595 | Jaccard: 0.6713
Epoch 1/3 |  val  | Loss: 1.6322 | Jaccard: 0.7195
Epoch 2/3 | train | Loss: 1.5929 | Jaccard: 0.7210
Epoch 2/3 |  val  | Loss: 1.6388 | Jaccard: 0.7158
Epoch 3/3 | train | Loss: 1.5069 | Jaccard: 0.7328
Epoch 3/3 |  val  | Loss: 1.6546 | Jaccard: 0.7208
Fold: 6
Epoch 1/3 | train | Loss: 2.0796 | Jaccard: 0.6708
Epoch 1/3 |  val  | Loss: 1.6544 | Jaccard: 0.7078
Epoch 2/3 | train | Loss: 1.5919 | Jaccard: 0.7180
Epoch 2/3 |  val  | Loss: 1.6450 | Jaccard: 0.7173
Epoch 3/3 | train | Loss: 1.4351 | Jaccard: 0.7392
Epoch 3/3 |  val  | Loss: 1.6319 | Jaccard: 0.7206
Fold: 7
Epoch 1/3 | train | Loss: 2.1183 | Jaccard: 0.6655
Epoch 1/3 |  val  | Loss: 1.5971 | Jaccard: 0.7187
Epoch 2/3 | train | Loss: 1.6268 | Jaccard: 0.7120
Epoch 2/3 |  val  | Loss: 1.5684 | Jaccard: 0.7219
Epoch 3/3 | train | Loss: 1.4952 | Jaccard: 0.7317
Epoch 3/3 |  val  | Loss: 1.5916 | Jaccard: 0.7231
Fold: 8
Epoch 1/3 | train | Loss: 2.1916 | Jaccard: 0.6603
Epoch 1/3 |  val  | Loss: 1.6748 | Jaccard: 0.7050
Epoch 2/3 | train | Loss: 1.6450 | Jaccard: 0.7120
Epoch 2/3 |  val  | Loss: 1.6062 | Jaccard: 0.7196
Epoch 3/3 | train | Loss: 1.5097 | Jaccard: 0.7304
Epoch 3/3 |  val  | Loss: 1.6520 | Jaccard: 0.7165
Fold: 9
Epoch 1/3 | train | Loss: 2.1056 | Jaccard: 0.6662
Epoch 1/3 |  val  | Loss: 1.7136 | Jaccard: 0.7122
Epoch 2/3 | train | Loss: 1.6063 | Jaccard: 0.7147
Epoch 2/3 |  val  | Loss: 1.6593 | Jaccard: 0.7206
Epoch 3/3 | train | Loss: 1.4619 | Jaccard: 0.7339
Epoch 3/3 |  val  | Loss: 1.6499 | Jaccard: 0.7212
Fold: 10
Epoch 1/3 | train | Loss: 2.0821 | Jaccard: 0.6699
Epoch 1/3 |  val  | Loss: 1.6264 | Jaccard: 0.7104
Epoch 2/3 | train | Loss: 1.5766 | Jaccard: 0.7236
Epoch 2/3 |  val  | Loss: 1.6017 | Jaccard: 0.7184
Epoch 3/3 | train | Loss: 1.4345 | Jaccard: 0.7424
Epoch 3/3 |  val  | Loss: 1.6217 | Jaccard: 0.7186

'''
import re 
import numpy as np

scores = re.findall(r'0\.\d{4}', text)
scores = [float(i) for i in scores]
np.mean(scores)

0.7105